In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# TL;DR

| Model                                       | R-squared (train)  | R-squared (val)    | R-squared (test)   | R-absolute (train) | R-absolute (val)   | R-absolute (test)  |
|---------------------------------------------|--------------------|--------------------|--------------------|--------------------|--------------------|--------------------|
| Linear regression                           | 0.5169866          |                    | 0.4549708          | 0.3383185          |                    | 0.3255524          |
| Lasso                                       | 0.5166027          |                    | 0.4613136          | 0.3371221          |                    | 0.3272954          |
| LARS                                        | 0.5160043          |                    | 0.4642462          | 0.3359979          |                    | 0.3276234          |
| Polynomial regression                       | 0.6010396          |                    | 0.5301241          | 0.3994169          |                    | 0.3740960          |
| Linear regression with RFE (6 features)     | 0.5160676          |                    | 0.4575793          | 0.3379842          |                    | 0.3255046          |
| Polynomial regression with RFE (5 features) | 0.4552918          |                    | 0.4691225          | 0.2864168          |                    | 0.3326174          |
| CrossNet                                    | 0.6884830          | 0.2226251          | 0.5017266          | 0.4879999          | 0.1206011          | 0.3968694          |
| MLP                                         | 0.9980898          | 0.1031491          | 0.2870642          | 0.9613156          | 0.1044721          | 0.2413247          |
| XGBoost                                     | 0.6163958          | 0.3052738          | 0.4813085          | 0.4093728          | 0.1779834          | 0.3262125          |
| EBM                                         | 0.6947590          | 0.4140548          | 0.4582469          | 0.4723313          | 0.2118888          | 0.3200194          |
| NAM                                         | 0.4631733          | 0.4038129          | 0.4546912          | 0.2804932          | 0.1968498          | 0.2907707          |
| NODE GAM                                    | 0.6060284          | 0.6050312          | 0.4252885          | 0.4231943          | 0.3740149          | 0.3055096          |
| NODE GA2M                                   | 0.4647184          | 0.4057951          | 0.3696757          | 0.3209596          | 0.3050901          | 0.2833377          |
| HONAM                                       | 0.9932572          | -1.1341181         | -0.7910015         | 0.9315096          | -0.3565330         | -0.2370180         |


# TODO in report
- add y transformation in methodology
- add info about batch size

- https://hastie.su.domains/Papers/LARS/LeastAngle_2002.pdf
- http://rstudio-pubs-static.s3.amazonaws.com/358961_ddc67a63195d43ed981b81f5bed43884.html
- https://strimmerlab.github.io/software/care/download/diabetes.pdf
- http://garthtarr.github.io/mplot/articles/diabetes.html
- https://github.com/federico-trotta/predicting_diabetes_with_ML
- LARS


In [2]:
import numpy as np
import pandas as pd
import torch
from utils import evaluate
from outliers import outliers_box

## Dataset

[Link to dataset](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html)

In [ ]:
df = pd.read_csv('./data/diabetes_data.csv')
df.head()

In [ ]:
df.describe()

In [ ]:
df.applymap(lambda x: x == 0).sum()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(20, 15))

mask = np.triu(np.ones_like(df.corr()))
ax = plt.axes()
sns.heatmap(df.corr(), annot=True, fmt='.2f', mask=mask, cmap="RdYlGn", vmin=-1, vmax=1, )
ax.set_title('Correlation matrix for raw diabetes progression data')
plt.show()

In [ ]:
import plotly.express as px
fig = px.scatter(df, x="tc", y="ldl", color='y', color_continuous_scale='rdylgn', labels={'tc': 'total serum cholesterol', 'ldl': 'low-density lipoprotein', 'y': 'disease progression 1 year after baseline'},
                 title='Correlation between low-density lipoprotein (ldl) and total serum cholesterol (tc)', height=1200, width=2400, trendline='ols')
fig.show()

In [ ]:
import plotly.express as px
fig = px.scatter(df, x="tch", y="hdl", color='y', color_continuous_scale='rdylgn', labels={'tch': 'total cholesterol', 'hdl': 'high-density lipoprotein', 'y': 'disease progression 1 year after baseline'},
                 title='Correlation between high-density lipoprotein (hdl) and total cholesterol (tch)', height=1200, width=2400, trendline='ols', trendline_options={'log_x': True, 'log_y': True})
fig.show()

In [ ]:
import plotly.figure_factory as ff
fig = ff.create_scatterplotmatrix(df, diag='histogram', size=3, title='Pairwise correltation in raw diabetes progression data', height=1200, width=2400)
for fig_data in fig.data:
    fig_data['marker']['color'] = '#1f77b4'
fig.show()

### Quantile transformation comparison

In [ ]:
from plotly.subplots import make_subplots

from sklearn.preprocessing import QuantileTransformer, StandardScaler
import plotly.figure_factory as ff

transformer = QuantileTransformer(output_distribution='normal', n_quantiles=df.shape[0])
scaler = StandardScaler()
var = 'y'
trans = transformer.fit_transform(df[[var]])
scaled = scaler.fit_transform(df[[var]])
trans_scaled = scaler.fit_transform(trans)

fig = ff.create_distplot([scaled.ravel(), trans_scaled.ravel()], ['Without Quantile Transformation', 'With Quantile Transformation'], show_hist=False, bin_size=0.01, show_curve=True, show_rug=False)
fig.update_layout(title_text=f'{var}')
fig.update_yaxes(range=[0, 1])
fig.update_xaxes(range=[-5, 5])
fig.show()

# Data pre-processing

In [3]:
from load_data import load_diabetes_data
columns, (x_train, y_train), (x_val, y_val), (x_test, y_test) = load_diabetes_data()
print('COLUMNS: ', columns)
print('training set: ', x_train.shape, y_train.shape)
print('validation set: ', x_val.shape, y_val.shape)
print('test set: ', x_test.shape, y_test.shape)

COLUMNS:  ['age', 'sex', 'bmi', 'map', 'tc', 'ldl', 'hdl', 'tch', 'ltg', 'glu']
training set:  (266, 10) (266, 1)
validation set:  (88, 10) (88, 1)
test set:  (88, 10) (88, 1)


In [4]:
task = 'regression'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batch_size = int(np.ceil(x_train.shape[0] / 100))
print(f'task={task}: device is {device} and batch size is {batch_size}')

task=regression: device is cpu and batch size is 3


In [5]:
from load_data import load_diabetes_data
from preprocessor import Preprocessor

preprocessor = Preprocessor(task=task, n_quantiles=x_train.shape[0])
x_train, y_train = preprocessor.fit_transform(x_train, y_train)
x_val, y_val = preprocessor.transform(x_val, y_val)
x_test, y_test = preprocessor.transform(x_test, y_test)

# Baselining

## Statsmodel

In [6]:
labels = {
  "age": "age",
  "sex": "sex",
  "bmi": "body mass index",
  "map": "average blood pressure",
  "tc": "total serum cholesterol",
  "ldl": "low-density lipoprotein",
  "hdl": "high-density lipoprotein",
  "tch": "total cholesterol",
  "ltg": "lamotrigine",
  "glu": "glucose",
  "y": "disease progression one year after baseline"
}

In [7]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def get_local_interpretation(y, pred_func, feature_weights):
    index = np.where(y_test==y)[0][0]
    x = x_test[index]
    feature_influence = x * feature_weights
    y_hat = pred_func(x)
    zeros = np.zeros(10)
    zeros[1] = np.NaN
    x_mean = preprocessor.inverse_transform_x([zeros])[0]
    x_true = preprocessor.inverse_transform_x([x])[0]
    y_true = preprocessor.inverse_transform_y([[y]])[0]
    y_pred = preprocessor.inverse_transform_y([[y_hat]])[0]
    y_mean = preprocessor.inverse_transform_y([[0]])[0]
    df = pd.DataFrame(np.stack([
        np.concatenate((x, [y], [y_hat])),
        np.concatenate((x_true, y_true, y_pred)),
        np.concatenate((x_mean, y_mean, y_mean)),
        np.concatenate((feature_weights, [np.NaN], [np.NaN])),        
        np.concatenate((feature_influence, [np.NaN], [np.NaN])),        
    ]), columns= [labels[c] for c in columns] + ['actual y', 'predicted y']).T.reset_index()
    df.columns = ['variable', 'scaled', 'original', 'mean_values', 'feature_weights', 'feature_contribution']
    
    fig = make_subplots(rows=1, cols=2, shared_yaxes=True, subplot_titles=("Feature contribution","Values of features and output relative to the mean"))

    fig.add_trace(go.Bar(x=df['feature_contribution'][:-2], y=df['variable'][:-2], orientation='h', text=df['variable'][:-2], showlegend=False, marker={'color':'#1f77b4'}), row=1, col=1)
    fig.add_trace(go.Bar(x=df['scaled'], y=df['variable'], orientation='h', text=df['variable'], showlegend=False, marker={'color':['#1f77b4' for i in range(10)] + ['#bcbd22', '#2ca02c']}),row=1, col=2)
    fig.add_trace(go.Bar(x=df['feature_weights'][:-2], y=df['variable'][:-2], orientation='h', showlegend=False, marker={'color':'#17becf'}),row=1, col=2)

    fig.update_layout(title_text="Local interpretation", height=800)
    fig.update_yaxes(visible=False, showticklabels=False)
    fig['layout']['xaxis1'].update(range=[-1,1], showgrid=False)
    fig['layout']['xaxis2'].update(range=[-5,5], showgrid=False)
    fig['layout'].update(barmode='relative')

    return fig

## Linear regression

In [8]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(x_train, y_train)
lr_fit = lr.predict(x_train)
lr_pred = lr.predict(x_test)

evaluate(y_train, lr_fit, task)
evaluate(y_test, lr_pred, task)
lr_weights = pd.DataFrame([(col, coefficient) for col, coefficient in zip(columns, lr.coef_[-1])], columns=['variable', 'coefficient'])
lr_weights['variable'] = lr_weights['variable'].apply(lambda x: labels[x])
lr_weights.sort_values(by='coefficient', key=abs, ascending=False)

R-squared: 0.5333725, R-absolute: 0.3554372
R-squared: 0.3856881, R-absolute: 0.2744805


,variable,coefficient
6,high-density lipoprotein,-0.390568
5,low-density lipoprotein,-0.365813
4,total serum cholesterol,0.333791
2,body mass index,0.255513
3,average blood pressure,0.235495
1,sex,-0.187449
8,lamotrigine,0.168010
0,age,0.061250
9,glucose,0.025344
7,total cholesterol,-0.021880


In [9]:
import plotly.express as px
fig = px.bar(lr_weights, y='variable', x='coefficient', orientation='h', range_x=(-0.5, 0.5), text='variable', title='Global weights for linear regression', height=800, width=800, labels={'coefficient': 'feature importance'})
fig.update_yaxes(visible=False, showticklabels=False)
fig.update_xaxes(showgrid=False)
fig.show()

In [10]:
fig = get_local_interpretation(np.percentile(y_test, 5, method='closest_observation'), lambda x: lr.predict([x])[0][0], lr.coef_[-1])
fig.show()

In [11]:
fig = get_local_interpretation(np.percentile(y_test, 95, method='closest_observation'), lambda x: lr.predict([x])[0][0], lr.coef_[-1])
fig.show()

## Lasso regression

## LARS

## Polynomial regression

## Linear regression with recursive feature elimination

## Polynomial regression with recursive feature elimination

## Comparison of all linear models

### Experiment configuration
- linear/logistic regression
- CrossNet (Wang et al. 2017)
  - number of units: 32
- XGBoost (Chen and Guestrin 2016)
  - number of boosting rounds: 1000
  - eta: 0.3
- EBM (Nori et al. 2019)
  - number of boosting rounds: 20000
  - inner bags: 8
  - outer bags: 8
  - learning rate: 0.01
- MLP
  - hidden layers: 3
  - hidden units: [32, 64, 32]
  - activation function: LeakyReLU
- NAM (Agarwal et al. 2020)
  - hidden layers: 3
  - hidden units: [32, 64, 32]
  - activation function: LeakyReLU
- NodeGAM (Chang, Caruana, and Goldenberg 2022), NodeGA2M
  - default hyperparameters provided by (Chang, Caruana, and Goldenberg 2022)
- HONAM
  - hidden layers: 3
  - hidden units: [32, 64, 32]
  - activation function: LeakyReLU
  
- epochs: 1000
- learning rate: 0.001
- batch size = (number of data samples)/100
- trained with 5 random seeds
- mean scores with standard deviations reported

## CrossNet
[deepctr-torch](https://deepctr-torch.readthedocs.io/en/latest/index.html)

In [117]:
from deepctr_torch.inputs import DenseFeat
from deepctr_torch.models import DCN

feature_columns = [DenseFeat(col, 1) for col in columns]

train_model_input = dict(zip(columns, x_train.T))
val_model_input = dict(zip(columns, x_val.T))
test_model_input = dict(zip(columns, x_test.T))
    
dcn = DCN(linear_feature_columns=feature_columns, dnn_feature_columns=feature_columns, dnn_hidden_units=[16, 32, 16], dnn_dropout=0.5, l2_reg_linear=0.0001, l2_reg_embedding=0.0001, l2_reg_cross=0.0001, l2_reg_dnn=0.00001, task='regression', device=device)
dcn.compile("adam", "mse", metrics=['mse'])
dcn.fit(train_model_input, y_train, validation_data=(val_model_input, y_val), epochs=1000)

cpu
Train on 266 samples, validate on 88 samples, 2 steps per epoch


2it [00:00, 164.42it/s]


Epoch 1/1000
0s - loss:  11.1428 - mse:  6.3944 - val_mse:  8.6055


2it [00:00, 128.85it/s]


Epoch 2/1000
0s - loss:  10.5330 - mse:  6.3076 - val_mse:  8.2640


...

Epoch 998/1000
0s - loss:  0.3533 - mse:  0.2268 - val_mse:  0.8978


2it [00:00, 122.62it/s]

Epoch 999/1000
0s - loss:  0.3532 - mse:  0.3055 - val_mse:  0.8974



2it [00:00, 118.79it/s]


Epoch 1000/1000
0s - loss:  0.3533 - mse:  0.3482 - val_mse:  0.8979


In [118]:
evaluate(y_train, dcn.predict(train_model_input), task)
evaluate(y_val, dcn.predict(val_model_input), task)
evaluate(y_test, dcn.predict(test_model_input), task)

R-squared: 0.6470730, R-absolute: 0.4515753
R-squared: 0.1883819, R-absolute: 0.0957897
R-squared: 0.4062193, R-absolute: 0.2926656


## Multi Layer Perceptron
[Tutorial](https://machinelearningmastery.com/building-multilayer-perceptron-models-in-pytorch/)

In [16]:
from sklearn.neural_network import MLPRegressor
# {'alpha': 0.0001,
#  'learning_rate': 'constant',
#  'learning_rate_init': 0.001,
#  'tol': 0.001
# }
mlp = MLPRegressor(hidden_layer_sizes=(32,64,32), batch_size=batch_size, learning_rate='adaptive', learning_rate_init=0.001, max_iter=1000, early_stopping=True, validation_fraction=0.2, tol=0.001, alpha=0.0001)
mlp.fit(np.concatenate((x_train, x_val)), np.concatenate((y_train, y_val)).ravel())

MLPRegressor(batch_size=3, early_stopping=True, hidden_layer_sizes=(32, 64, 32),
             learning_rate='adaptive', max_iter=1000, tol=0.001,
             validation_fraction=0.2)

In [17]:
evaluate(y_train, mlp.predict(x_train), task)
evaluate(y_val, mlp.predict(x_val), task)
evaluate(y_test, mlp.predict(x_test), task)

R-squared: 0.5850241, R-absolute: 0.3732409
R-squared: 0.5314549, R-absolute: 0.3753710
R-squared: 0.4614782, R-absolute: 0.3196569


## XGBoost
[dlmc](https://xgboost.readthedocs.io/en/stable/index.html)

In [50]:
from xgboost import XGBRegressor
xgb = XGBRegressor(n_estimators=1000, learning_rate=0.01, gamma=1.5, max_depth=3, objective='reg:squarederror', eval_metric='rmse')
xgb.fit(x_train, y_train)

evaluate(y_train, xgb.predict(x_train), task)
evaluate(y_val, xgb.predict(x_val), task)
evaluate(y_test, xgb.predict(x_test), task)
pd.DataFrame([(col, coefficient) for col, coefficient in zip(columns, xgb.feature_importances_)], columns=['variable', 'feature_importance']).set_index(['variable']).sort_values(by=['feature_importance'], ascending=False)

R-squared: 0.7072534, R-absolute: 0.4590703
R-squared: 0.4158109, R-absolute: 0.3029248
R-squared: 0.4664185, R-absolute: 0.3028730


,feature_importance
variable,
bmi,0.290631
ltg,0.223089
tch,0.073904
glu,0.072161
tc,0.072156
map,0.071237
ldl,0.059353
hdl,0.057093
age,0.041938


## Explainable Boosting Machine
[interpret.ml](https://interpret.ml/docs/ebm.html#)

In [ ]:
from interpret.glassbox import ExplainableBoostingRegressor
ebm = ExplainableBoostingRegressor(outer_bags=8, inner_bags=8, learning_rate=0.01, max_rounds=30_000)
ebm.fit(x_train, y_train)
ebm_pred = ebm.predict(x_test)

evaluate(y_train, ebm.predict(x_train), task)
evaluate(y_val, ebm.predict(x_val), task)
evaluate(y_test, ebm.predict(x_test), task)

In [ ]:
from interpret import show
show(model.explain_global())

## NAM
- [google-research/neural_additive_models](https://github.com/google-research/google-research/tree/master/neural_additive_models)
- [emeln/nam](https://github.com/lemeln/nam)

In [ ]:
from nam.wrapper import NAMRegressor
nam = NAMRegressor(num_epochs=1000, num_learners=10, metric='mse', monitor_loss=True, n_jobs=-1, random_state=42, val_split=0.2)
nam.fit(np.concatenate((x_train, x_val)), np.concatenate((y_train, y_val)).ravel())

In [ ]:
evaluate(y_train, nam.predict(x_train), task)
evaluate(y_val, nam.predict(x_val), task)
evaluate(y_test, nam.predict(x_test), task)

In [ ]:
import plotly.figure_factory as ff
fig = ff.create_distplot([y_test.ravel(), nam.predict(x_test).ravel()], ['actual', 'predicted'], show_hist=False)
fig.show()

## NODE GAM
[github](https://github.com/zzzace2000/nodegam)

In [ ]:
from nodegam.sklearn import NodeGAMRegressor

nodegam = NodeGAMRegressor(len(columns), validation_size=0.2, ga2m=0, device=device)
nodegam.fit(pd.DataFrame(x_train_eval, columns=columns), y_train_eval.ravel())

In [ ]:
evaluate(y_train, nodegam.predict(pd.DataFrame(x_train, columns=columns)), task)
evaluate(y_val, nodegam.predict(pd.DataFrame(x_val, columns=columns)), task)
evaluate(y_test, nodegam.predict(pd.DataFrame(x_test, columns=columns)), task)

## NODE GA2M

In [ ]:
from nodegam.sklearn import NodeGAMRegressor

nodega2m = NodeGAMRegressor(len(columns), validation_size=0.2, ga2m=1, device=device)
nodega2m.fit(pd.DataFrame(x_train_eval, columns=columns), y_train_eval.ravel())

In [ ]:
evaluate(y_train, nodega2m.predict(pd.DataFrame(x_train, columns=columns)), task)
evaluate(y_val, nodega2m.predict(pd.DataFrame(x_val, columns=columns)), task)
evaluate(y_test, nodega2m.predict(pd.DataFrame(x_test, columns=columns)), task)

## HONAM

In [ ]:
from model import HONAM
honam = HONAM(num_features=x_train.shape[1], out_size=1, task=task, order=2, verbose=False).to(device)
honam.fit(x_train, y_train, x_val, y_val)

In [ ]:
evaluate(y_train, honam.predict(x_train), task)
evaluate(y_val, honam.predict(x_val), task)
evaluate(y_test, honam.predict(x_test), task)

In [ ]:
x_test[0].reshape(-1,1).dtype

In [ ]:
honam.forward(torch.from_numpy(x_test[0].reshape(-1,1)))

In [ ]:
import plotly.figure_factory as ff
fig = ff.create_distplot([y_test.ravel(), honam.predict(x_test).ravel()], ['actual', 'predicted'], show_hist=False)
fig.show()

In [ ]:
abs(-1)

In [ ]:
y_i = 100
r2 = []
r_abs = []
x = []
for y_hat in range(-50, 251):
    r2.append(1 - ((y_i - y_hat)**2 / y_i**2))
    r_abs.append(1 - (abs(y_i - y_hat) / y_i))
    x.append(y_hat)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=r2, mode='lines', name='R-squared'))
fig.add_trace(go.Scatter(x=x, y=r_abs, mode='lines', name='R-absolute'))
fig.add_vline(x=100, line_dash="dash", annotation_text="actual value", annotation_position="bottom right")
fig.update_layout(title='Change in scores with different prediction errors', xaxis_title='predicted value', yaxis_title='score', height=800)


fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=r_abs[:151], y=r2[:151], mode='lines', name='R-squared'))
# fig.update_layout(title='Change in scores with different prediction errors', xaxis_title='predicted value', yaxis_title='score')
fig.show()